In [ ]:
# default_exp convert

# The Converter

> The internals for the lib2nbdev functionality

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from fastcore.test import *

In [ ]:
#export
from fastcore.basics import Path
from fastcore.foundation import Config
from fastcore.script import call_parse

from fastprogress.fastprogress import progress_bar

from nbdev.export import nbglob, export_names, _re_class_func_def, _re_obj_def
from nbdev.sync import _split

## Foundational Helper Functions

In [ ]:
#export
def start_cell() -> str:
    """
    Returns a blank Jupyter cell
    """
    return '''{
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],'''

A very simplistic and foundational function, it simply returns a string representation of a blank Jupyter cell without any metadata. Any information that should be filled in needs to be centralized to `outputs`

In [ ]:
#hide
_default_cell = '''{
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],'''
test_eq(_default_cell, start_cell())

In [ ]:
#export
def init_nb(module_name:str) -> str:
    """
    Initializes a blank notebook based on `module_name`

    Also writes the first #default_exp cell and checks for a nested module (moduleA.moduleB)
    """
    if module_name[0] == '.': module_name = module_name.split('.')[1]
    if '.ipynb' in module_name: module_name = module_name.split('.ipynb')[0]
    return '{\n "cells": [' + start_cell() + f'''
   "source": [\n    "# default_exp {module_name}"
   ''' + ']\n  },'

In [ ]:
#hide
_initial_nb = '{\n "cells": [{\n   "cell_type": "code",\n   "execution_count": null,\n   "metadata": {},\n   "outputs": [],\n   "source": [\n    "# default_exp testname"\n   ]\n  },'
test_eq(_initial_nb, init_nb("testname"))

In [ ]:
#export
def write_cell(code:str, is_public:bool=False) -> str:
    """
    Takes source `code`, adds an initial #export tag, and writes a Jupyter cell
    """
    if is_public is None: export = ''
    export = '#export' if is_public else '#exporti'
    cell = start_cell() + \
    f'''
    "source": [\n    "{export}\\n",\n'''
    arr = code.split('\n')
    for c in arr[:-1]:
        c = c.replace('"', '\\"').replace('/', '//')
        cell += f'    "{c}\\n",\n'
    last = arr[-1].replace('"', '\\"').replace('/', '//')
    cell += f'    "{last}"'
    return cell + ']},'

This function will write a cell given some `code` (which is a str). `is_public` is there to determine if `#export` or `#exporti` should be used (a public or private function, class, or object). 

In [ ]:
#export
def finish_nb() -> str:
    """
    Writes the last remaining metadata for finishing a Jupyter notebook
    """
    return ''']}],
     "metadata": {
  "jupytext": {
   "split_at_heading": true
  },
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
'''

In [ ]:
#export
def write_nb(splits:list, num:int, parent:str=None, private_list:list=[]) -> str:
    """
    Writes a fully converted Jupyter Notebook based on `splits` and saves it in `Config`'s `nbs_path`.

    The notebook number is based on `num`

    `parent` denotes if the current notebook module is based on a parent module
    such as `moduleA.moduleB`
    
    `private_list` is a by-cell list of `True`/`False` for each block of code of whether it is private or public
    """
    # Get filename
    fname = splits[0][0]
    if fname[0] == '.': fname = fname[1:]
    if parent is not None: fname = f'{parent}.{fname}'

    # Initialize and write notebook
    nb = init_nb(fname)
    for i, (_, code) in enumerate(splits):
        c = write_cell(code, private_list[i])
        nb += c
    nb = nb[:-3] + finish_nb()

    # Figure out the notebook number
    if num < 10:
        fname = f'0{num}_{fname}'
    else:
        fname = f'{num}_{fname}'

    # Save notebook in `nbs_path`
    with open(f'{Config().path("nbs_path")/fname}', 'w+') as source_nb:
        source_nb.write(nb)

In [ ]:
#exporti
def _not_private(n):
    "Checks if a func is private or not, alternative to nbdev's"
    for t in n.split('.'):
        if (t.startswith('_') and not t.startswith('__')): return False
    return '\\' not in t and '^' not in t and t != 'else'

In [ ]:
#export
@call_parse
def convert_lib():
    """
    Converts existing library to an nbdev one by autogenerating notebooks.
    
    Basic prerequisites:
      - You must have made a nbdev settings.ini file beforehand
      - Optionally you can add `# Cell` and `# Internal Cell` tags in the source files where you would like specific cells to be
    
    Run this command in the base of your repo
    
    **Can only be run once**
    """
    print('Gathering files...')
    files = nbglob(extension='.py', config_key='lib_path', recursive=True)
    if len(files) == 0: raise ValueError("No files were found, please ensure that `lib_path` is configured properly in `settings.ini`")
    print(f'{len(files)} modules found in the library')
    num_nbs = len(files)
    nb_path = Config().path('nbs_path')
    nb_path.mkdir(exist_ok=True)
    print(f'Writing notebooks to {nb_path}...')
    if nb_path.name == Config().lib_name:
        nb_path = Path('')
        slash = ''
        
    else:
        nb_path = Path(nb_path.name)
        slash = '/'

    for num, file in enumerate(progress_bar(files)):
        if (file.parent.name != Config().lib_name) and slash is not None:
            parent = file.parent.name
        else:
            parent = None
        fname = file.name.split('.py')[0] + '.ipynb'
        if fname[0] == '.': fname = fname[1:]
        # Initial string in the .py
        init_str = f"# AUTOGENERATED! DO NOT EDIT! File to edit: {nb_path}{slash}{fname} (unless otherwise specified).\n\n"

        # Override existing code to include nbdev magic and one code cell
        with open(file, encoding='utf8') as f: code = f.read()

        if "AUTOGENERATED" not in code:
            code = init_str + code

        # Check to ensure we haven't tried exporting once yet
        if "# Cell" and "# Internal Cell" not in code and '__all__' not in code:
            split_code = code.split('\n')
            private_list = []
            _do_pass, _private, _public = False, '# Internal Cell\n', '# Cell\n'
            for row, line in enumerate(split_code):
                if _do_pass: _do_pass = False; continue
                # Deal with decorators
                if '@' in line:
                    code = split_code[row+1]
                    if code[:4] == 'def ': code = code[4:]
                    if 'patch' in line or 'typedispatch' in line or not line[0].isspace():
                        is_private = _not_private(code.split('(')[0])
                        private_list.append(is_private)
                        split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
                    _do_pass = True
                # Deal with objects
                elif _re_obj_def.match(line) and not _do_pass:
                    is_private = _not_private(line.split('(')[0])
                    private_list.append(is_private)
                    split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
                # Deal with classes or functions
                elif _re_class_func_def.match(line) and not _do_pass:
                    is_private = _not_private(line.split(' ')[1].split('(')[0])
                    private_list.append(is_private)
                    split_code[row] = f'{_public}{line}' if is_private else f'{_private}{line}'
            code = '\n'.join(split_code)

        # Write to file
        with open(file, 'w', encoding='utf8') as f: f.write(code)

        # Build notebooks
        splits = _split(code)
        write_nb(splits, num, parent, private_list)
        
        # Generate the `__all__` in the top of each .py
        if '__all__' not in code:
            c = code.split("(unless otherwise specified).")
            code = c[0] + "(unless otherwise specified).\n" + f'\n__all__ = {export_names(code)}\n\n# Cell' + c[1]
            with open(file, 'w', encoding='utf8') as f: f.write(code)
    print(f"{Config().lib_name} successfully converted!")